In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import math

def haversine_distance(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth specified in decimal degrees using the Haversine formula.
    """
    # Convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])

    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

def calculate_s_values(positions):
    """
    Calculate normalized cumulative distances (s values) for a list of positions.
    Each position is a tuple (latitude, longitude).
    """
    distances = []
    cumulative_distances = [0]  # Start with 0 for the first position

    # Calculate distances between consecutive points
    for i in range(1, len(positions)):
        lat1, lon1 = positions[i - 1]
        lat2, lon2 = positions[i]
        dist = haversine_distance(lat1, lon1, lat2, lon2)
        
        distances.append(dist)
        cumulative_distances.append(cumulative_distances[-1] + dist)

    # Normalize cumulative distances
    total_distance = cumulative_distances[-1]
    
    # s_values = [cd / total_distance for cd in cumulative_distances]

    return cumulative_distances



In [12]:
df, summary = pd.read_pickle('Teconer_results/drawn-wildflower-1028.pkl')


df['UnixTime'] = df['UnixTime'] - df['UnixTime'].min()
# calculate trip station using calculate_s_values
df['trip_station'] = calculate_s_values(df[['Latitude', 'Longitude']].values)
# group by trip_id and make each group a separate df
grouped = df.groupby('TripID')

trip_dfs = [group for _, group in grouped]

# print average length of trip
trip_lengths = [len(trip) for trip in trip_dfs]

estimators = ['TMI']




# calculate the MAE and RMSE across all records in df
def calculate_error_metrics(df, estimator):
    """
    Calculate mean absolute error (MAE) and root mean squared error (RMSE) between Friction (meansured) and predicted value by each estimator.
    """
    # Calculate mean absolute error
    mae = np.mean(np.abs(df['Friction (measured)'] - df[estimator]))

    # Calculate root mean squared error
    rmse = np.sqrt(np.mean((df['Friction (measured)'] - df[estimator]) ** 2))

    # Calculate R^2
    ss_res = np.sum((df['Friction (measured)'] - df[estimator]) ** 2)
    ss_tot = np.sum((df['Friction (measured)'] - np.mean(df['Friction (measured)'])) ** 2)
    r2 = 1 - ss_res / ss_tot

    return mae, rmse, r2

# print the summary dictionary make sure each item appears in a separate line
for key, value in summary.items():
    print(key, value)
for estimator in estimators:

    print("Estimator: ", estimator)
    mae, rmse, r2 = calculate_error_metrics(df, estimator)
    print(f"MAE: {mae:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"R2: {r2:.2f}")




FileNotFoundError: [Errno 2] No such file or directory: 'Teconer_results/drawn-wildflower-1028.pkl'

In [9]:
%matplotlib qt

plt.close('all')
virgin = True
fig, ax = plt.subplots(2, 1, figsize=(10, 5))


for trip in trip_dfs:
    if len(trip) < 1:
        continue
    ax[0].cla()
    ax[1].cla()

    # scatter plot of friction in a lat long 2D space using plt
    sc = ax[0].scatter(trip['Longitude'], trip['Latitude'], c=trip['Friction (measured)'], cmap='viridis', s=3, alpha=0.7, vmin=.6, vmax=.8)
    ax[0].set_title('Measured Friction')
    ax[0].set_xlabel('Longitude')
    ax[0].set_ylabel('Latitude')
    ax[0].grid(True)


    if virgin:
        virgin = False
        # add colorbar to the scatter plot
        fig.colorbar(sc, ax=ax[0], label='Friction')
        fig.tight_layout()

    # plot friction over time using plt
    ax[1].plot(trip['trip_station'], trip['Friction (measured)'], label='Measured')
    for method_name in estimators:
        ax[1].plot(trip['trip_station'], trip[method_name], label=method_name)
    # ax[2].set_title('Measured vs. Predicted - '+str(trip_id) + ' - ' + str(trip[1][0,0]))
    ax[1].set_title('Friction (Measured vs. Predicted)')
    ax[1].legend()
    ax[1].set_xlabel('Trip Distance [Km]')
    ax[1].set_ylabel('Friction')
    ax[1].grid(True)

    # make ylim .2 to 08
    ax[1].set_ylim(0, 1)

    plt.tight_layout()
    plt.pause(0.01)

    # save the png figure in \figures directory
    # plt.savefig('figures/fig_'+str(trip_id)+'_kswin.png')
    # if key n is pressed, go to next trip
    if plt.waitforbuttonpress():            
        continue

KeyboardInterrupt: 